<a href="https://colab.research.google.com/github/Tauntybird/CIS700/blob/master/my_game_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text adventure game

This Python notebook builds a simple text advenutre game inspired by the [Adventuron Classroom](https://adventuron.io/classroom/) design by Chris Ainsley of Adventuron Software Limited.

The main components are:
1. __The parser__, which interprets the player's commands.
2. __The game__, which represents the world (a collection of __locations__ and __items__), and describes what the player sees.
3. __The data__, which you input to create your own unique game.

## The Game Class
The game keeps track of the state of the world, and describes what the player sees as they move through different locations.

In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    self.curr_location.has_been_visited = True
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # Print the special commands associated with items in the game (helpful 
    # for debugging and for novice players).
    self.print_commands = True
    # extra variables for story order
    self.met_lion = False
    self.see_lion_trapped = False
    self.befriend_elephant = False
    self.poacher_left = False
    self.net_cut = False

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.name)
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)
        if self.print_commands:
          special_commands = item.get_commands()
          for cmd in special_commands:
            print('\t', cmd)

  def add_to_inventory(self, item):
    """Add an item to the player's inventory."""
    self.inventory[item.name] = item
  
  def is_in_inventory(self,item):
    return item.name in self.inventory

  def get_items_in_scope(self):
    """Returns a list of items in the current location and in the inventory"""
    items_in_scope = []
    for item_name in self.curr_location.items:
      items_in_scope.append(self.curr_location.items[item_name])
    for item_name in self.inventory:
      items_in_scope.append(self.inventory[item_name])
    return items_in_scope

## Locations

Locations Locations are the places in the game that a player can visit.  They contain connects to other locations and items that the player can interact with.

In [0]:
class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections is a dictionary whose keys are directions and
     whose values are the location that is the result of traveling in that 
     direction.  The travel_descriptions also has directions as keys, and its 
     values are an optional short desciption of traveling to that location.
  """
  def __init__(self, name, description, end_game=False):
    # A short name for the location
    self.name = name
    # A description of the location
    self.description = description
    # True if entering this location should end the game
    self.end_game = end_game
    # Dictionary mapping from directions to other Location objects
    self.connections = {}
    # Dictionary mapping from directions to text description of the path there
    self.travel_descriptions = {}
    # Dictionary mapping from item name to Item objects present in this location
    self.items = {}
    # Dictionary mapping from direction to Block object in that direction
    self.blocks = {}
    # Flag that gets set to True once this location has been visited by player
    self.has_been_visited = False

  def add_connection(self, direction, connected_location, travel_description=""):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    self.travel_descriptions[direction] = travel_description
    if direction == 'north':
      connected_location.connections["south"] = self
      connected_location.travel_descriptions["south"] = ""
    if direction == 'south':
      connected_location.connections["north"] = self
      connected_location.travel_descriptions["north"] = ""
    if direction == 'east':
      connected_location.connections["west"] = self
      connected_location.travel_descriptions["west"] = ""
    if direction == 'west':
      connected_location.connections["east"] = self
      connected_location.travel_descriptions["east"] = ""
    if direction == 'up':
      connected_location.connections["down"] = self
      connected_location.travel_descriptions["down"] = ""
    if direction == 'down':
      connected_location.connections["up"] = self
      connected_location.travel_descriptions["up"] = ""
    if direction == 'in':
      connected_location.connections["out"] = self
      connected_location.travel_descriptions["out"] = ""
    if direction == 'out':
      connected_location.connections["in"] = self
      connected_location.travel_descriptions["in"] = ""


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)


  def is_blocked(self, direction, game):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
        return False
    (block_description, preconditions) = self.blocks[direction]
    if check_preconditions(preconditions, game):
      # All the preconditions have been met.  You may pass.
      return False
    else: 
      # There are still obstalces to overcome or puzzles to solve.
      return True

  def get_block_description(self, direction):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
      return ""
    else:
      (block_description, preconditions) = self.blocks[direction]
      return block_description

  def add_block(self, blocked_direction, block_description, preconditions):
    """Create an obstacle that prevents a player from moving in the blocked 
       location until the preconditions are all met."""
    self.blocks[blocked_direction] = (block_description, preconditions)

## Checking Preconditions 
In text adventure games it's common to block a player's progress by creating blocks that prevent them from moving to a location.  For instance, a drawbridge might have a troll that you need to get rig of before you can cross into the castle, or a locked door might prevent you from entering a building until you have a key.  

This is a function that you can modify to include other preconditions.

In [0]:
def check_preconditions(preconditions, game, print_failure_reasons=True):
  """Checks whether the player has met all of the specified preconditions"""
  all_conditions_met = True
  for check in preconditions: 
    if check == "inventory_contains":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        if print_failure_reasons:
          print("You don't have the %s" % item.name)
    if check == "in_location":
      location = preconditions[check]
      if not game.curr_location == location:
        all_conditions_met = False
        if print_failure_reasons:
          print("You aren't in the correct location")
    if check == "location_has_item":
      item = preconditions[check]
      if not item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s isn't in this location" % item.name)
    if check == "location_does_not_have_item":
      item = preconditions[check]
      if item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s is in this location" % item.name)
  return all_conditions_met

## Items
Items are objects that a player can get, or scenery that a player can examine. We could also implement people as items.  

In [0]:
class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self,
               name,
               description,
               examine_text="",
               take_text="",
               start_at=None,
               gettable=True,
               end_game=False):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Text that displays when player takes an object.
    self.take_text = take_text if take_text else ("You take the %s." % self.name)
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # True if entering this location should end the game.
    self.end_game = end_game
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.commands = {}


  def get_commands(self):
    """Returns a list of special commands associated with this object"""
    return self.commands.keys()

  def add_action(self, command_text, function, arguments, preconditions={}):
    """Add a special action associated with this item"""
    self.commands[command_text] = (function, arguments, preconditions)

  def do_action(self, command_text, game):
    """Perform a special action associated with this item"""
    end_game = False  # Switches to True if this action ends the game.
    if command_text in self.commands:
      function, arguments, preconditions = self.commands[command_text]
      if check_preconditions(preconditions, game):
        end_game = function(game, arguments)
    else:
      print("Cannot perform the action %s" % command_text)
    return end_game

## Sentiment analysis



In [0]:
import nltk
nltk.download('punkt')
from textblob import TextBlob

## The Parser
The parser is the module that handles the natural language understanding in the game.  The players enter commands in text, and the parser interprets them and performs the actions that the player intends.  This is the module with the most potential for improvement using modern natural language processing.  The implementation that I have given below only uses simple keyword matching.

In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game

  def get_player_intent(self,command):
    command = command.lower()
    if "," in command:
      # Let the player type in a comma separted sequence of commands
      return "sequence"
    elif self.get_direction(command):
      # Check for the direction intent
      return "direction"
    elif command.lower() == "look" or command.lower() == "l":
      # when the user issues a "look" command, re-describe what they see
      return "redescribe"
    elif "examine " in command or command.lower().startswith("x "):
      return "examine"
    elif  "take " in command or "get " in command:
      return "take"
    elif "drop " in command:
      return "drop"
    elif "inventory" in command or command.lower() == "i":
      return "inventory"
    else: 
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return "special"

  def parse_command(self, command):
    # add this command to the history
    self.command_history.append(command)

    # By default, none of the intents end the game. The following are ways this
    # flag can be changed to True.
    # * Going to a certain place.
    # * Entering a certain special command
    # * Picking up a certain object.

    end_game = False

    # Intents are functions that can be executed
    intent = self.get_player_intent(command)
    if intent == "direction":
      end_game = self.go_in_direction(command)
    elif intent == "redescribe":
      self.game.describe()
    elif intent == "examine":
      self.examine(command)
    elif intent == "take":
      end_game = self.take(command)
    elif intent == "drop":
      self.drop(command)
    elif intent == "inventory":
      self.check_inventory(command)
    elif intent == "special":
      end_game = self.run_special_command(command)
    elif intent == "sequence":
      end_game = self.execute_sequence(command)
    else:
      # check if player is talking to the (awake) lion or the elephant
      if (self.game.curr_location.name == "Forest") and ("lion" in self.game.curr_location.items):
        blob = TextBlob(command)
        score = 0
        for sentence in blob.sentences:
          score = score + (sentence.sentiment.polarity)
        if score > 0:
          return self.run_special_command(destroy_item(self.game, (lion,"To your surprise and relief, the lion lets out a hearty laugh (it’s more like a growl but you can tell he’s amused) as he gently sets you back on the ground." +
                "\n'Alright, little mouse. I’ll let you live. Let’s see when or how you can repay me.' \nThe lion is still laughing to himself as he stalks away into the brush.")))
        elif score == 0:
          print("The lion seemed to ignore you. Try saying something else.")
        else:
          print("The lion roars in anger, rupturing your eardrums, before closing its fangs around you as you disappear down his gullet. \nYou die wishing you had said something nicer.")
          return True 
      # check if player is talking to zebras
      elif (self.game.curr_location.name == "Savanna"):
        blob = TextBlob(command)
        score = 0
        for sentence in blob.sentences:
          score = score + (sentence.sentiment.polarity)
          if score > 0:
            print("The zebras snort among each other. They seem pleased, by the way they are ruffling their noses at you.")
          elif score == 0:
            print("The zebras seemed to ignore you. Try saying something else.")
          else:
            print("The zebras whinny and stomp in annoyance, swishing their tails angrily.")
        return False
      # check if player is talking to giraffe
      elif (self.game.curr_location.name == "Meadow"):
        blob = TextBlob(command)
        score = 0
        for sentence in blob.sentences:
          score = score + (sentence.sentiment.polarity)
          if score > 0:
            print("You notice the giraffe’s ear twitch. She looks slightly pleased.")
          elif score == 0:
            print("The giraffe seemed to ignore you. Try saying something else.")
          else:
            print("The giraffe either doesn’t hear you or pretends not to.")
        return False
      else:
        print("I'm not sure what you want to do.")
    return end_game

  ### Intent Functions ###

  def go_in_direction(self, command):
    """ The user wants to in some direction """
    direction = self.get_direction(command)

    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction, self.game):
          # check to see whether that direction is blocked.
          print(self.game.curr_location.get_block_description(direction))
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]

          # If moving to this location ends the game, only describe the location
          # and not the available items or actions.
          if self.game.curr_location.end_game:
            self.game.describe_current_location()
          else:
            self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())
    return self.game.curr_location.end_game

  def check_inventory(self,command):
    """ The player wants to check their inventory"""
    if len(self.game.inventory) == 0:
      print("You don't have anything.")
    else:
      descriptions = []
      for item_name in self.game.inventory:
        item = self.game.inventory[item_name]
        descriptions.append(item.description)
      print("You have: ", end = '')
      print(*descriptions, sep = ", ",)
  

  def examine(self, command):
    """ The player wants to examine something """
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")

  def take(self, command):
    """ The player wants to put something in their inventory """
    command = command.lower()
    matched_item = False

    # This gets set to True if posession of this object ends the game.
    end_game = False

    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.add_to_inventory(item)
          self.game.curr_location.remove_item(item)
          print(item.take_text)
          end_game = item.end_game
        else:
          print("You cannot take the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

    return end_game

  def drop(self, command):
    """ The player wants to remove something from their inventory """
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          matched_item = True
          item = self.game.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")


  def run_special_command(self, command):
    """Run a special command associated with one of the items in this location
       or in the player's inventory"""
    for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return item.do_action(special_command, self.game)

  def execute_sequence(self, command):
    for cmd in command.split(","):
      cmd = cmd.strip()
      self.parse_command(cmd)

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    if command.startswith("go out"):
      return "out"
    if command.startswith("go in"):
      return "in"
    for exit in self.game.curr_location.connections.keys():
      if command == exit.lower() or command == "go " + exit.lower():
        return exit
    return None

## Special functions
Many times we want to add special behavior to items in the game.  For instance, we might want to be able to _pick a rose_ from a _rosebush_, or the _eat_ a _fish_.  In this implementation we do this in a pretty generic way by allowing the game developer to call ```Item.add_action(cmd,function,argment,preconditions)``` where ```function``` is any Python function. Some example of functions are defined below.

These functions should return True if the game is ended by the action, False otherwise.

In [0]:
def add_item_to_inventory(game, *args):
  """ Add a newly created Item and add it to your inventory."""
  (item, action_description, already_done_description) = args[0]
  if(not game.is_in_inventory(item)):
    print(action_description)
    game.add_to_inventory(item)
  else:
    print(already_done_description)
  return False

def describe_something(game, *args):
  """Describe some aspect of the Item"""
  (description) = args[0]
  print(description)
  return False

def perform_multiple_actions(game, *args):
  for function in args[0]:
    if function[0](game, function[1]):
      return True

def destroy_item(game, *args):
  """Removes an Item from the game by setting its location is set to None."""
  (item, action_description) = args[0]
  if game.is_in_inventory(item):
    game.inventory.pop(item.name)
    print(action_description)
  elif item.name in game.curr_location.items:
    game.curr_location.remove_item(item)
    print(action_description)
  else:
    print(already_done_description)
  return False

def remove_action(game, *args):
  (item, action) = args[0]
  item.commands.pop(action)
  return False

def befriend(game, *args):
  (location, item) = args[0]
  location.remove_item(item)
  game.befriend_elephant = True
  return False

def cut_net(game, *args):
  description = args[0]
  if not game.befriend_elephant:
    print("The poacher sees you trying to cut through the net and yells so loud in anger that he scares you to death.")
    return True
  game.net_cut = True
  print("It takes some time, but eventually you create a hole large enough for the lion to escape through. \nDuring this entire time, the lion is still fast asleep and shows no sign of waking. If you don’t find a way to rouse the lion before the poacher returns, he will just be recaptured. \nThere must be a way to wake him...")
  return False

def end_game(game, *args):
  """Ends the game."""
  end_message = args[0]
  print(end_message)
  return True

## Game Data

Here's where you can define the locations and items in your game.  To get you started, I defined a super-simple fishing game, which contains the first 3 locations of __Action Castle__ by Jared A. Sorensen, which is part of the awesome book [Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf).  

You can play through the whole game with the following commands:
1. take pole
2. go out
3. south 
4. catch fish with pole
5. eat fish

In [0]:
# Locations
savanna = Location("Savanna", "A vast savanna scattered with shrubs and isolated trees. A heavy rainfall had just passed, leaving the savanna floor muddy and damp. You can try talking to the zebras.")	
poacher_camp = Location( "Poacher's camp", "A flat patch of grass hidden by some tall trees. A shabby tent is set up nearby, along with some leftover wood and ashes from a fire. \nThe front of the tent seems to have a small hole near the bottom.")
canyon = Location("Canyon", "A deep canyon with steep, rocky cliffs. Small rocks and pebbles constantly roll off the walls. It seems to run for many more miles.")
waterhole = Location("Waterhole", "The small waterhole is a popular meeting spot for nearby animals, though there doesn’t seem to be many here now. \nSmall trees and bushes surround the area. The water looks unusually clear.") 
meadow = Location("Meadow", "An unkempt field of grass spotted with patches of flowers. \nVines slightly disrupt the peaceful look as they hungrily search for more pieces of land to expand to. You can try talking to the giraffe.") 
forest = Location("Forest", "As you enter the quiet forest, your ears alert you to some bushes rustling behind you. Instantly your adrenaline kicks in. \nBut before you can run to safety, something huge bursts out of the bushes and leaps onto you, pinning you beneath sharp claws. \nBefore you can even squeak, your attacker picks you up roughly by pinching your tail (ouch!). Your stomach drops to the ground as you stare into two large golden orbs. \nThe lion is staring at you curiously. He looks slightly disappointed. \n'Oh, it’s just a mouse. I was hoping for something a bit bigger for my next meal.' \nWhat do you have to say?")
tent = Location("Tent", "The poacher sees you run across the camp. Unfortunately, he has steel-toe boots. \nHe kicks you and you slam into a nearby tree trunk. Stunned, you can’t react as he picks you up and roughly throws you into a small cage. \nHe throws a sheet over it and you never see the light again.", end_game=True)

# Items needed to be global
lion = Item("lion", "a lion", "A regal golden lion. You are around the height of one of his claws. By the glint in his eye, you can tell he is looking for his next meal.", start_at=forest, gettable=False)
sleeping_lion = Item("lion (asleep)", "an asleep lion", "A golden lion. He’s deep asleep and trapped beneath a heavy net. He looks a bit less regal like this.", start_at=None, gettable=False)
net = Item("net", "a net", "A large net braided with thick, heavy ropes.", start_at=None)


In [0]:
def build_game():

  # Connections
  savanna.add_connection("east", poacher_camp)
  poacher_camp.add_connection("east", canyon)
  poacher_camp.add_connection("south",  waterhole)
  poacher_camp.add_connection("north", meadow)
  poacher_camp.add_connection("in", tent)
  meadow.add_connection("north", forest)

  # Charaters
  elephant = Item("elephant", "a baby elephant", "A baby elephant. Even as a baby, he is much much larger than you. You’d say he looks intelligent... but then again, all elephants are.", start_at=waterhole, gettable=False)
  poacher = Item("poacher", "a poacher", "A green-clothed human. He has shifty eyes and always keeps a hand on a tool he carries, which appears to be a long ranged weapon of sorts.", start_at=poacher_camp, gettable=False)
  zebras = Item("zebras", "a herd of zebras", "A herd of zebras. You can’t tell where one zebra starts and ends.", start_at=savanna, gettable=False)
  giraffe = Item("giraffe", "a giraffe", "A lone giraffe. She’s a little tall to be your type.", start_at=meadow, gettable =False)

  # Items that you can pick up
  rock = Item("rock", "a sharp rock", "A sharp, jagged piece of rock. It seems like it chipped off the canyon walls.", start_at=canyon)
  red_berries = Item("red berries", "red berries", "A bunch of juicy, red berries. They are protected by thorny stems and leaves, so larger animals can’t easily eat them.", start_at=forest)
  purple_berries = Item("purple berries", "purple berries", "A bunch of juicy, purple berries. They are protected by thorny stems and leaves, so larger animals can’t easily eat them.", start_at=forest)
  vial = Item("vial", "a vial of liquid", "A vial of tinted liquid. You’re not the best at reading English, but you think the label says drinking the liquid will turn you into a large animal or can awaken a large animal.",start_at=tent)
  red_berries_action = Item("red berries (action)", "red berries", "", start_at=None)
  purple_berries_action = Item("purple berries (action)", "purple berries", "", start_at=None)
  flowers = Item("flowers", "some flowers", "The flowers come in a multitude of dainty pastel colors.", start_at=meadow)
  flower_crown = Item("flower crown", "a flower crown", "A beautiful flower crown.", start_at=None)

  # Imagery
  water = Item("water", "a waterhole", "You take a peek at the surface of the water. It’s perfectly smooth and reflects the gorgeous blue sky and wispy white clouds. \nYou see a stunningly attractive mouse staring back at you. What’s cookin’, good lookin’?", start_at=waterhole, gettable=False)

  # Add special functions to your items
  elephant.add_action("talk to elephant", describe_something, 
                      ("You scamper up to the baby elephant. You make sure to approach him where he can see you so he doesn’t accidentally trample you. \nHe’s shedding large tears as he cries. " + 
                       "\n'Where is my momma? I miss her so much… She said she’d be back soon... I’m getting really hungry... I wish I had some red berries to eat right now...' \nYour heart goes out to the baby elephant." + 
                       "Maybe if you get some red berries for him to eat, he will cheer up and talk to you?"))
 # lion.add_action("insult lion", end_game, ("You retort that for such a dumb, lousy lion, a small meal like yourself is more than enough to nourish their tiny brain and scrawny build. \nThe lion roars in anger, rupturing your eardrums," +
 #                                           "before closing its fangs around you as you disappear down his gullet. "))
 # lion.add_action("beg for mercy", destroy_item, (lion, "You desperately plead with the lion to spare your life, offering to do him any favor you can in the future. \nTo your surprise and relief, the lion lets out a hearty laugh" +
 #                                                 "(it’s more like a growl but you can tell he’s amused) as he gently sets you back on the ground. \n'Alright, little mouse. I’ll let you live. Let’s see when or how you can repay me.' \nThe lion is still laughing to himself as he stalks away into the brush."))
  elephant.add_action("give red berries to elephant", perform_multiple_actions, 
                                [(destroy_item, (red_berries_action, "The elephant looks at you hopefully. \n'Red berries!' he cries happily. 'Did you get these for me?' \nYou nod and push them towards him. " +
                                                 "He gingerly picks them up with his trunk and eats them one by one, savoring their taste. \nAfter he is done, he looks rejuvenated and turns to you in excitement. \n'Thank you so much! I’m so happy to have met a mouse like you! " +
                                                 "If you ever need my help with anything, just ask!' \nYou explain to the elephant about the lion’s capture, and the elephant agrees to help. He will trumpet and lead the poacher to the waterhole away from the lion.")), 
                                 (remove_action, (elephant, "talk to elephant")), (befriend, (poacher_camp, poacher))], preconditions={"in_location":waterhole, "inventory_contains": red_berries_action})
  red_berries_action.add_action("eat red berries", describe_something, ("These are the baby elephant's favorite."))
  elephant.add_action("give purple berries to elephant", destroy_item, (purple_berries_action, "The elephant looks at you sadly. \n'Oh, purple berries… I wish they were the red ones... "+
                                                                                     "Thank you for bringing me these, but you can have them...'"), preconditions={"in_location":waterhole, "inventory_contains": purple_berries_action})
  purple_berries_action.add_action("eat purple berries", end_game, ("These berries were poisonous! "))
  red_berries.add_action("pick red berries", perform_multiple_actions, [(add_item_to_inventory, (red_berries_action, "You pick some red berries.", "You already have red berries.")), (destroy_item, (red_berries, ""))])
  purple_berries.add_action("pick purple berries", perform_multiple_actions, [(add_item_to_inventory, (purple_berries_action, "You pick some purple berries.", "You already have purple berries.")), (destroy_item, (purple_berries, ""))]) 
  poacher.add_action("hit poacher with rock", end_game, "You muster up all of your bloodlust and take a swing at the poacher. \nUnfortunately, he sees you and kills you with one gunshot.", preconditions={"inventory_contains": rock, "in_location":poacher_camp})
  vial.add_action("drink vial", end_game, ("You start gulping down the liquid from the vial. \nYour heart starts beating faster as you feel your blood course through your body. Are you transforming?! \nBut seconds later you start to feel dizzy. Your heart can’t keep up! \nYour hearing is overtaken by a high ringing noise and your vision blurs before fading away..."), preconditions={"inventory_contains": vial})
  vial.add_action("give vial to lion", end_game, ("As soon as you drip the last bit of liquid from the vial into the lion’s maw, the lion lets out a huge yawn that shakes the ground beneath you. \nHe wakes up and stretches luxuriously, before looking around in confusion. He takes in the poacher’s tent, the cut net, and the empty vial, before focusing his attention on you." +
                                                  "\n'Did… did you just save me?'\nYou nod, feeling smug inside.\nThe lion shakes his head in disbelief. 'Well well, I guess you really did follow through with your promise.' \nHe offers you a claw, which you carefully wrap your paw around as you shake it. The lion now looks at you with newfound respect.\n'I will never forget you, mouse. If we cross paths again, I will not try to turn you into a meal again. This act of kindness will be with me forever.' \nBefore you can respond, the lion leaps over you and runs into the forest. \nYou smile, feeling a glow of pride inside, before also scampering off into the forest.\nTHE END. YOU WIN!"), preconditions={"in_location":poacher_camp, "inventory_contains": vial, "location_does_not_have_item":net})
  # zebras.add_action("talk to zebras", describe_something, "You approach the herd of zebras and try to talk to them. \nUnfortunately, you quickly realize the zebras are too dumb to make intelligent conversation with, so you give up.", preconditions={"in_location":savanna})
  # giraffe.add_action("talk to giraffe", describe_something, "You approach the lone giraffe and try to talk to her. \nHowever, the altitude difference is too great, and she is too preoccupied with munching on tree leaves to notice your lowly existence.",preconditions={"in_location":savanna})
  flowers.add_action("pick flowers", add_item_to_inventory, (flowers, "You carefully pick some flowers, but don’t know what you plan to do with them.", "You already picked some flowers."), preconditions={"in_location":meadow})
  flowers.add_action("smell flowers", describe_something, "The flowers smell amazing. You want to rub them all over your fur.", preconditions={"in_location":meadow})
  flowers.add_action("make flower crown", add_item_to_inventory, (flower_crown, "You take the flowers you picked earlier and weave them into a small flower crown. It’s so beautiful you shed a tear.", "You already have a flower crown."), preconditions={"inventory_contains":flowers})
  flower_crown.add_action("wear flower crown", destroy_item, (flower_crown, "You put on the flower crown you made. It’s not particularly useful for anything, but it makes you feel special and beautiful. "), preconditions={"inventory_contains":flower_crown})

  game = Game(savanna)
  net.add_action("cut net with rock", perform_multiple_actions, [(destroy_item, (net, "You take the rock and carefully slice through the individual ropes of the net.")), 
                                                                 (cut_net, "")], preconditions={"inventory_contains":rock, "in_location":poacher_camp})

  return Game(savanna)


# Play the game
This small snippet of code is what you need to run the game.  Behold! The magestic prompt! 

In [0]:
def game_loop():
  print("Welcome to A SMALL ACT OF KINDNESS")
  print("You are a cute, little dormouse exploring the great big world.")
  game = build_game()

  # reset variables
  game.met_lion = False
  game.see_lion_trapped = False
  game.befriend_elephant = False
  game.poacher_left = False
  game.net_cut = False
  if "lion (asleep)" in poacher_camp.items:
    del poacher_camp.items["lion (asleep)"]
  if "net" in poacher_camp.items:
    del poacher_camp.items["net"]
  forest.description = "As you enter the quiet forest, your ears alert you to some bushes rustling behind you. Instantly your adrenaline kicks in. \nBut before you can run to safety, something huge bursts out of the bushes and leaps onto you, pinning you beneath sharp claws. \nBefore you can even squeak, your attacker picks you up roughly by pinching your tail (ouch!). Your stomach drops to the ground as you stare into two large golden orbs. \nThe lion is staring at you curiously. He looks slightly disappointed. \n'Oh, it’s just a mouse. I was hoping for something a bit bigger for my next meal.' \nWhat do you have to say?"
  poacher_camp.description = "A flat patch of grass hidden by some tall trees. A shabby tent is set up nearby, along with some leftover wood and ashes from a fire. \nThe front of the tent seems to have a small hole near the bottom."
  

  parser = Parser(game)
  game.describe()
  command = ""
  while not (command.lower() == "exit" or command.lower == "q"):
    command = input(">")
    end_game = parser.parse_command(command)
    if end_game:
      return
    if game.curr_location == forest:
      if not game.met_lion:
        forest.add_block("south", "You must respond to the lion", preconditions={"location_does_not_have_item": lion})
        game.met_lion = True
        poacher_camp.items["lion (asleep)"] = sleeping_lion
        poacher_camp.items["net"] = net
        forest.description = "The forest’s full canopy allows adequate light to pass down to the flat, fertile grounds below. Thin branches droop from trees, \nand a range of plants grow across the forest floor. A medley of animal sounds resonate through the air."
        poacher_camp.description = "Upon entering the clearing, you immediately notice something off. \nThe poacher is pacing around his camp closer than before, and his eyes are scanning the vicinity intently while also flicking down occasionally to check on a golden pile. \nUpon closer inspection, you are shocked to realize it’s the lion! He’s fast asleep and trapped under a heavy net."
    if game.curr_location == poacher_camp:
      if (not game.see_lion_trapped) and game.met_lion:
        game.see_lion_trapped = True
        poacher_camp.description = "A flat patch of grass hidden by some tall trees. A shabby tent is set up nearby, along with some leftover wood and ashes from a fire. \nThe front of the tent seems to have a small hole near the bottom. The lion is trapped in a large net."
    if game.befriend_elephant and (not game.poacher_left):
      poacher_camp.description = "Just as you enter the clearing, a lone elephant trumpet could be heard not far from camp, in the direction of the waterhole. \nYou watch intently as the poacher hesitates, looking back at the sleeping lion, before deciding to run off in the direction of the waterhole. \nThe camp is now left undefended."
      tent.end_game = False
      tent.description = "An unrolled sleeping bag is sprawled across the middle of the floor. Books and papers are scattered around it. \nA small, unzipped pouch with its contents spilled out in the corner of the tent catches your eye."
      if game.curr_location == poacher_camp:
        game.poacher_left = True
        poacher_camp.description = "The poacher is still no where in sight."
game_loop()
print('THE GAME HAS ENDED.')

Welcome to A SMALL ACT OF KINDNESS
You are a cute, little dormouse exploring the great big world.
Savanna
A vast savanna scattered with shrubs and isolated trees. A heavy rainfall had just passed, leaving the savanna floor muddy and damp. You can try talking to the zebras.
Exits: East
You see: 
a herd of zebras
>e
Poacher's camp
['A flat patch of grass hidden by some tall trees. A shabby tent is set up nearby, along with some leftover wood and ashes from a fire. \nThe front of the tent seems to have a small hole near the bottom.']
Exits: West, East, South, North, In
You see: 
a poacher
	 hit poacher with rock
>n
Meadow
An unkempt field of grass spotted with patches of flowers. 
Vines slightly disrupt the peaceful look as they hungrily search for more pieces of land to expand to. You can try talking to the giraffe.
Exits: South, North
You see: 
a giraffe
some flowers
	 pick flowers
	 smell flowers
	 make flower crown


# Visualize your game
The code below allows you to create a directed graph that shows the locations in your game and how they are connected.  You can also save a PDF of your graph to your Google Drive with the `save_to_drive` method.  The output file will be called `game-visualization.pdf`.

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue

def DFS(game, graph):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""
  start_location = game.curr_location
  frontier = queue.Queue()
  frontier.put(start_location)
  visited = {}
  visited[start_location.name] = True

  while not frontier.empty():
    current_location = frontier.get()
    game.curr_location = current_location
    name = current_location.name
    description = current_location.description
    items = current_location.items
    items_html = describe_items(current_location)
    html = "<<b>%s</b><br />%s<br />%s>" % (name, description, items_html)
    # Create a new node in the graph for this location
    graph.node(name, label=html)  

    connections = current_location.connections
    for direction in connections.keys():
      next_location = connections[direction]
      if not current_location.is_blocked(direction, game):
        # Create an edge between the current location and its successor
        graph.edge(name, next_location.name, label=direction.capitalize())
      else:
        # Create a dotted edge for connected locations that are blocked
        block_description = "%s\n%s" % (direction.capitalize(), current_location.get_block_description(direction))
        graph.edge(name, next_location.name, label=block_description, style="dotted")
      if not next_location.name in visited:
        visited[next_location.name] = True
        frontier.put(next_location)

def describe_items(location, print_commands=True):
    """Describe what objects are in the current location."""
    items_html = ""
    if len(location.items.keys()) > 0:
      items_html = "You see: "
    for item_name in location.items:
      item = location.items[item_name]
      items_html += item.description
      if print_commands:
        special_commands = item.get_commands()
        for cmd in special_commands:
          items_html += "<br/><i>%s</i>" % cmd
    return items_html

def save_to_drive(graph):
  from google.colab import drive
  drive.mount('/content/drive/')
  graph.render('/content/drive/My Drive/game-visualization', view=True)  

graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
game = build_game()
DFS(game, graph)
#save_to_drive(graph)
graph
